In [7]:
pip install yellowbrick

Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 282.6/282.6 kB 2.1 MB/s eta 0:00:00a 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [9]:
import bs4
import os
import re

import pandas as pd
import sklearn as sk

from sklearn.metrics import *
from bs4 import BeautifulStoneSoup
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
import numpy as np
from imblearn.over_sampling import SMOTE
from sklearn.preprocessing import MinMaxScaler
from sklearn import preprocessing
from sklearn.tree import DecisionTreeClassifier
from catboost import CatBoostClassifier, Pool
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn import tree
from catboost.utils import get_confusion_matrix
from yellowbrick.contrib.wrapper import wrap
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from yellowbrick.classifier import ConfusionMatrix
import matplotlib.pyplot as plt
from sklearn.impute import SimpleImputer

random_state_all = 1   # фиксирование случайного состояния, чтобы была воспроизводимость результата

In [10]:
def parser(path : str) -> list:
    filenames = [html for html in os.listdir(path) if '.html' in html]      # Собираем имена файлов с разрешением html в папке, переданной в параметр path
    messages_all = []                                                       # Переменная для хранения сообщений

    for file in filenames:                                                  # Перебираем имена файлов, собранных ранее
        with open(f'{path}\{file}', "rb") as html_file:                      # Открываем файл на чтение
            index = html_file.read()                                        # Записываем содержимое файла (html-разметку страницы) в переменную

        page = bs4.BeautifulSoup(index)                                     # Создаем объект bs4, позволяющий парсить html

        messages = page.find_all('div', class_='text')                      # Собираем все элементы div с классом text (сообщения) в список

        for name in messages:                                               # Перебираем каждое сообщение
            messages_all.append(name.text.strip())                          # Дополняем список всех сообщений чата взятым на этой итерации сообщением с убранными пробелами по краям
    
    return(messages_all)                                                    # Возвращаем результат работы

In [14]:

yacht_info = parser('/Users/peregrine146/Downloads/yachts/yachts.html')
avto_info = parser('/Users/peregrine146/Downloads/avto/')

NotADirectoryError: [Errno 20] Not a directory: '/Users/peregrine146/Downloads/yachts/yachts.html'

In [ ]:
avia_history

['Основной аэродром 🇺🇦',
 'Привет, дорогой читатель!Скорее всего тебе доводилось летать на современном Боинге или Аэробусе. Сейчас это кажется обычным делом, а ведь всего 114 лет назад братья Райт совершили первый полёт на своём Flyer I. А первый реактивный пассажирский лайнер de Havilland Comet поднялся в небо в 1951 году. Что было между этими событиями? Кто создал первый реактивный самолёт? Чьи авиаконструкторы создавали самые странные самолёты? Первые трансатлантические перелёты и их герои. Авиация Золотого века. Экспериментальные летательные аппараты. Первые авиакомпании и их пассажиры. Полярная авиация.Обо всём этом и многом другом читайте здесь.От винта! ✈️',
 'В октябре 1910 года посетители Международного авиационного салона в Париже увидели очень необычный самолёт - без пропеллера! Так мир познакомился с первым самолётом на реактивной тяге - Coandă-1910.',
 'Промо-брошюра конструктора Coandă-1910 - румынского инженера Анри Коандэ. "Только самолёты без пропеллеров".',
 'Coandă-1

In [ ]:
avia_info

['FlightMode',
 'Всем привет! Если вы любите гражданскую авиацию! Вам интересно узнать как живут, работают и отдыхают пилоты, бортпроводники, диспетчера и техники. Если вы хотите быть в курсе последних событий об авиации в России и за рубежом! То вам на наш канал! Самое новое, свежее и интересное из мира самолётов, аэропортов и людей, связанных с этой сферой - здесь! Полетели! 🛫',
 'Начнём с новостей из московского Внуково. Там в районе девяти утра чуть не случился транспортный коллапс. Причина - неработающая багажная лента. Из-за сбоя в работе программного обеспечения разгружать и загружать багаж пришлось вручную! Пассажиры ждут отправления, самолёты ждут вылета. Не, ну а что, не лететь же с пустыми руками в отпуск. В авиакомпании "Россия", которая является одним из базовых перевозчиков аэропорта сообщили, что ситуация полностью под контролем и серьёзных задержек не произошло. Выдыхаем!',
 'Вам сосиску с макаронами или гречу с курой?Если вы любите вкусно поесть на борту самолета, то э

In [ ]:
yacht_info


NameError: name 'yacht_info' is not defined

In [ ]:
aviainfo_df = pd.DataFrame(avia_info, columns = ['text']).dropna().drop_duplicates()   # Создаем датафрейм из данных парсера со столбцом text, удаляем пустые значения и дубликаты
aviainfo_df['y'] = avia_info[0]                                                        # Первая строка - название канала.
aviainfo_df['text'] = aviainfo_df['text'].apply(lambda x: re.sub('[^\w\s#@/:%.,!?_-]', '', x))  # Замена символов, формирующих смайлики, на пустоту
aviainfo_df = pd.DataFrame(aviainfo_df.loc()[1:])                                               # Убираем первую строку из датафрейма

In [ ]:
aviahistory_df = pd.DataFrame(avia_history, columns = ['text']).dropna().drop_duplicates()   # Создаем датафрейм из данных парсера со столбцом text, удаляем пустые значения и дубликаты
aviahistory_df['y'] = avia_history[0]                                                        # Первая строка - название канала.
aviahistory_df['text'] = aviahistory_df['text'].apply(lambda x: re.sub('[^\w\s#@/:%.,!?_-]', '', x))  # Замена символов, формирующих смайлики, на пустоту
aviahistory_df = pd.DataFrame(aviahistory_df.loc()[1:])                                               # Убираем первую строку из датафрейма

In [ ]:
avia_df = pd.concat([aviahistory_df, aviainfo_df], ignore_index=True)                    # Создаем объединенный датафрейм по сообщениям из двух городов

In [ ]:
avia_df

,text,y
0,"Привет, дорогой читатель!Скорее всего тебе дов...",Основной аэродром 🇺🇦
1,В октябре 1910 года посетители Международного ...,Основной аэродром 🇺🇦
2,Промо-брошюра конструктора Coandă-1910 - румын...,Основной аэродром 🇺🇦
3,"Coandă-1910 выполнил один единственный полёт, ...",Основной аэродром 🇺🇦
4,"Идеи, лежащие в основе конструкции Coandă-1910...",Основной аэродром 🇺🇦
...,...,...
9185,Стоимость билетов за рубеж пришла в норму посл...,FlightMode
9186,Росавиация считает себя Министерством гражданс...,FlightMode
9187,"Самолёт Airbus, собранный в Китае, поставлен ...",FlightMode
9188,Emirates хочет летать в российские регионыАвиа...,FlightMode


Передаю мешку слов датафрейм с сообщениями без изменений, поскольку токенизация происходит внутри библиотеки. Если попытаться сделать тоекнизацию и отдать на преобразование векторизатору датафрейм, где в каждой строке столбца text будет массив токенов, то будет выдана ошибка. Объяснения есть ниже.

Выдержка из комментария к библиотеке:

input : {'filename', 'file', 'content'}, default='content'

        * If `'filename'`, the sequence passed as an argument to fit is
          expected to be a list of filenames that need reading to fetch
          the raw content to analyze.

        * If `'file'`, the sequence items must have a 'read' method (file-like
          object) that is called to fetch the bytes in memory.

        * If `'content'`, the input is expected to be a sequence of items that
          can be of type string or byte.

Изо всех вариантов выше видно, что нам максимально подходит только третий, где передаваемая информация должна быть итерируемым массивом из элементов типа строк и байтов.

In [ ]:
X, y = avia_df.text, avia_df.y                                                                                  # Создаем Х и У из столбцов датафрейма
X_train, X_test, y_train, y_test = sk.model_selection.train_test_split(X, y, test_size=0.35, random_state=random_state_all) # Разделяем датафрейм на тренировочную и тестовую выборки

vectorizer = CountVectorizer(lowercase = True)                              # Создаем обхект векторизации методом мешка слов с параметром, переводящим текст в нижний регистр
X_train_counts = vectorizer.fit_transform(X_train)                          # Преобразовываем тренировочные фичи в вектора

clf = MultinomialNB()                                                       # Создаем объект Ниавного Байесовского классификатора
clf.fit(X_train_counts, y_train)                                            # Тренируем классификатор на тренироваочных данных ( y не преобразовали, поскольку это лишь указание класса, которому принадлежит каждая строка )

score = clf.score(vectorizer.transform(X_test), y_test)                     # Оцениваем точность модели на тестовых данных ( так же преобразовали фичи и не тронули метки )

new_text = [' '.join("""
В 1930-х годах грузовые самолёты обычно представляли собой переделанные пассажирские. Это накладывало свои ограничения на погрузочно-разгрузочные операции. Для реализации своих военных планов #Германия нуждалась в новом, изначально разработанном для перевозки грузов, самолёте. Весной 1939 года был объявлен конкурс, в котором приняла участие и компания Arado Flugzeugwerke. В итоге её проект Ar 232 Tausendfüßler был признан победителем.

Это был высокоплан с погрузочной рампой в задней части фюзеляжа и двухкилевым хвостовым оперением. Шасси было двух видов — трёхстоечное для аэродромов с нормальной ВПП и 22-колёсное для полевых аэродромов. Версия Ar 232B оснащалась 4 × двигателями BMW Bramo 323R-2 Fafnir (986 л.с.) Крейсерская скорость — 290 км/ч, дальность полёта — 1062 км. Брал на борт до 4500 кг груза. Вооружение — 20-мм пушка + 3 × 13-мм пулемёта + до 8 × 7,92-мм пулемётов при перевозке солдат.

К моменту начала эксплуатации Ar 232 в 1943 году, Германия уже не могла позволить выделять много ресурсов на новые проекты. В итоге построили всего 20 самолётов в двух версиях Ar 232A (двухмоторная) и Ar 232B (четырёхмоторная). 2 самолёта после Второй мировой войны какое-то время эксплуатировались Королевскими ВВС Великобритании.""".split('\n'))] # В этой переменной храним текст из одной из двух групп для проверки его принадлежности этим группам
predicted_test = clf.predict(vectorizer.transform(X_test))                  # Предсказываем, каким классам ( группам ) принадлежать сообщения из тестовой выборки
predicted = clf.predict(vectorizer.transform(new_text))[0]                  # Предсказываем какому классу ( какой группе ) принадлежит введенный выше текст

# Преобразовываем объекты Series в датафреймы
y_test_df = pd.DataFrame(y_test).reset_index().drop('index', axis = 1)
y_test_predicted_df = pd.DataFrame(predicted_test.tolist()).rename(columns = {0 : 'y'})


# Выводим оценки качества модели и предсказанный класс
print(f"Точность: {score}\nf1-мера: {f1_score(y_test_df, y_test_predicted_df, pos_label=y_test_df.loc()[0].tolist()[0])}")                                     
print(f"Предсказание введенного текста: {predicted}")

Точность: 0.9738887161952129
f1-мера: 0.9664
Предсказание введенного текста: Основной аэродром 🇺🇦


In [ ]:
sm = SMOTE(random_state=42, k_neighbors=5)
X_res, y_res = sm.fit_resample(X, y)

X_train, X_test, y_train, y_test = train_test_split(X_res, y_res, test_size=0.33, random_state=42)

ValueError: could not convert string to float: 'Привет, дорогой читатель!Скорее всего тебе доводилось летать на современном Боинге или Аэробусе. Сейчас это кажется обычным делом, а ведь всего 114 лет назад братья Райт совершили первый полёт на своём Flyer I. А первый реактивный пассажирский лайнер de Havilland Comet поднялся в небо в 1951 году. Что было между этими событиями? Кто создал первый реактивный самолёт? Чьи авиаконструкторы создавали самые странные самолёты? Первые трансатлантические перелёты и их герои. Авиация Золотого века. Экспериментальные летательные аппараты. Первые авиакомпании и их пассажиры. Полярная авиация.Обо всём этом и многом другом читайте здесь.От винта! '